In [ ]:
import torch
from IPython.display import Audio, display
from chunkformer_vpb.training.finetune_config import FinetuneConfig
from chunkformer_vpb.training.data_loader import VivosDataset, collate_fn_entry
from torch.utils.data import DataLoader

# === Load config và dataset ===
cfg = FinetuneConfig.from_yaml("../../config/aug_vpb_ctc_finetune_config.yaml")
train_ds = VivosDataset(cfg, "train")
# print lenght dataset 
print(f"🔍 Dataset length: {len(train_ds)} samples")

# === Tạo dataloader để iterate gốc (batch=1) ===
train_loader = DataLoader(train_ds, batch_size=1, shuffle=False, collate_fn=collate_fn_entry)

# === Bước 1: Lưu 10 sample gốc đầu tiên (có entry + waveform) ===
base_entries = []
entry_dict = {}  # map utt_id → (entry, wav) cho nhanh

# === Step 1: Build entry_dict toàn bộ
entry_dict = {}
full_loader = DataLoader(train_ds, batch_size=1, shuffle=False, collate_fn=collate_fn_entry)
for batch in full_loader:
    _, _, _, _, entries = batch
    entry, wav = entries[0]
    utt_id = entry.utt_id if hasattr(entry, "utt_id") else entry["utt_id"]
    entry_dict[utt_id] = (entry, wav)

# === Step 2: Lấy 10 bản gốc đầu tiên
base_entries = [
    (utt_id, entry, wav)
    for utt_id, (entry, wav) in entry_dict.items()
    if "_aug" not in utt_id
][:10]

# === Bước 2: Duyệt và phát audio gốc + từng bản augment đã load sẵn ===
print("\n🔁 Nghe các bản augment đã có trong dataset:\n")

for base_utt, base_entry, base_wav in base_entries:
    print(f"\n🧾 Gốc: {base_utt}")
    display(Audio(base_wav[0].numpy(), rate=cfg.data.sample_rate, autoplay=False))

    # Dò augment
    for suffix in ["aug1", "aug2", "aug3"]:
        aug_utt = f"{base_utt}_{suffix}"
        if aug_utt in entry_dict:
            aug_entry, aug_wav = entry_dict[aug_utt]
            aug_type = getattr(aug_entry, "augment_type", "❓")
            print(f"   ↳ 🌀 {aug_utt} | augment_type = {aug_type}")
            display(Audio(aug_wav[0].numpy(), rate=cfg.data.sample_rate, autoplay=False))
        else:
            print(f"   ⚠️ Không tìm thấy {aug_utt}")
    print("-" * 60)


/home/kylh/.local/share/mamba/envs/stt310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


../../../vpb_dataset
DataConfigFT(manifest_dir='../../../vpb_dataset/manifest_vpb_non_empty_full', audio_dir='../../../vpb_dataset', sample_rate=16000, num_mel_bins=80, frame_length=25, frame_shift=10, dither=0.0, energy_floor=0.0, train_meta_file='train_meta_augmented.json')
🔍 Dataset length: 13652 samples
